In [1]:
import torch
import torchvision
import glob
from PIL import Image, ImageOps
import onnxruntime
import numpy as np
import time


## 自作前処理　＋　onnx 

In [2]:
model = torch.hub.load('./hidden_others/yolov5', 'custom', 'best.onnx', source='local')

YOLOv5 🚀 2024-3-10 Python-3.8.18 torch-2.2.1+cu121 CPU

Loading best.onnx for ONNX Runtime inference...
Adding AutoShape... 


In [3]:
img_files = ['test_002.jpg']

In [4]:
# 処理された画像を保持するリスト
processed_imgs = []

img_size = 640

for img_file in img_files:
    # 画像を開き、期待するサイズにリサイズ
    img = Image.open(img_file)
    img.thumbnail((img_size, img_size))
    img = ImageOps.pad(img, size=((img_size, img_size)))
    img.save('pad_002.jpg')
    processed_imgs.append(img)

for img in processed_imgs:
    # model.conf = 0.6
    result = model(img)
    result.print()
    print(result.xyxy)

image 1/1: 640x640 1 押印, 1 未押印
Speed: 5.7ms pre-process, 344.9ms inference, 10.7ms NMS per image at shape (1, 3, 640, 640)


[tensor([[4.27367e+02, 4.45427e+02, 4.72278e+02, 5.03631e+02, 9.22218e-01, 1.00000e+00],
        [4.27713e+02, 4.46479e+02, 4.73168e+02, 5.03123e+02, 4.53981e-01, 0.00000e+00]])]


In [5]:
result.xyxy

[tensor([[4.27367e+02, 4.45427e+02, 4.72278e+02, 5.03631e+02, 9.22218e-01, 1.00000e+00],
         [4.27713e+02, 4.46479e+02, 4.73168e+02, 5.03123e+02, 4.53981e-01, 0.00000e+00]])]

## 自前前処理　＋　best.pt 

In [10]:
model1 = torch.hub.load('./hidden_others/yolov5', 'custom', './hidden_others/best.pt', source='local')

YOLOv5 🚀 2024-3-10 Python-3.8.18 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5m summary: 212 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [7]:
for img in processed_imgs:
    # model1.conf = 0.6
    result = model1(img)
    result.print()
    print(result.xyxy)

image 1/1: 640x640 1 押印, 1 未押印
Speed: 11.9ms pre-process, 409.9ms inference, 0.9ms NMS per image at shape (1, 3, 640, 640)


[tensor([[4.27367e+02, 4.45427e+02, 4.72278e+02, 5.03631e+02, 9.22218e-01, 1.00000e+00],
        [4.27713e+02, 4.46479e+02, 4.73168e+02, 5.03123e+02, 4.53982e-01, 0.00000e+00]])]


In [104]:
print(result.xyxy)

[tensor([[4.27367e+02, 4.45427e+02, 4.72278e+02, 5.03631e+02, 9.22218e-01, 1.00000e+00],
        [4.27713e+02, 4.46479e+02, 4.73168e+02, 5.03123e+02, 4.53982e-01, 0.00000e+00]])]


## yolov5 + best.pt

In [105]:
model1 = torch.hub.load('./hidden_others/yolov5', 'custom', './hidden_others/best.pt', source='local')

YOLOv5 🚀 2024-3-10 Python-3.8.18 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5m summary: 212 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [106]:
result = model1(img_files)
# model1.conf = 0.6
result.print()
result.xyxy

image 1/1: 1161x819 1 押印, 1 未押印
Speed: 63.9ms pre-process, 399.6ms inference, 1.1ms NMS per image at shape (1, 3, 640, 480)


[tensor([[6.04151e+02, 8.07996e+02, 6.85716e+02, 9.13076e+02, 9.32026e-01, 1.00000e+00],
         [6.01491e+02, 8.06746e+02, 6.88052e+02, 9.11995e+02, 7.41273e-01, 0.00000e+00]])]

In [107]:
result.pandas().xyxy

[         xmin        ymin        xmax        ymax  confidence  class name
 0  604.151001  807.996033  685.716492  913.076233    0.932026      1  未押印
 1  601.491211  806.746277  688.052124  911.995056    0.741273      0   押印]

In [9]:
def preprocess_image(image_path, target_size=640):
    image_list = []
    for one_image in image_path:
        image = Image.open(one_image)
        image.thumbnail((target_size, target_size))
        padded_image = ImageOps.pad(image, (target_size, target_size), color='black') 
        image_np = np.array(padded_image).astype(np.float32) / 255.0
        image_np = np.transpose(image_np, [2, 0, 1])  # HWCからCHWへ変換
        image_np = np.expand_dims(image_np, axis=0)  # バッチサイズの次元を追加
    return image_np

In [91]:
def preprocess_image(image_path, target_size=640):
    image_list = []
    for one_image in image_path:
        image = Image.open(one_image)
        image.thumbnail((target_size, target_size))
        padded_image = ImageOps.pad(image, (target_size, target_size), color='black') 
        image_np = ((np.array(padded_image).astype(np.float32) / 255.0)-0.5)*2
        image_np = np.transpose(image_np, [2, 0, 1])  # HWCからCHWへ変換
        image_np = np.expand_dims(image_np, axis=0)  # バッチサイズの次元を追加
    return image_np

In [92]:
# ONNX Runtimeセッションを作成し、モデルをロード
session = onnxruntime.InferenceSession('best.onnx')

In [93]:
image_path = img_files
input_tensor = preprocess_image(image_path)

In [94]:
# 推論を実行
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
result = session.run([output_name], {input_name: input_tensor})

In [95]:
conf_thres = 0.25  # 信頼値の閾値
iou_thres = 0.45  # NMS IOU の閾値
agnostic_nms = False
max_det = 1000


pred = torch.from_numpy(result[0])
pred = non_max_suppression(pred, conf_thres, iou_thres, agnostic_nms, max_det=max_det)

In [96]:
pred

[tensor([[4.27126e+02, 4.45251e+02, 4.72056e+02, 5.03764e+02, 9.26464e-01, 1.00000e+00],
         [4.27496e+02, 4.46230e+02, 4.73546e+02, 5.03092e+02, 4.41865e-01, 0.00000e+00]])]

#### conf_thresを下げると押印欄の検知がされる！！！！！！！！！！！！！！！！

In [16]:
def box_label (
    img,
    box,
    label ="",
    color = (56, 56, 255),
    txt_color = (255, 255, 255),
    
):
    """
    オリジナル画像にクラス、クラス確率のテキストが付与されたbboxを描画する。

    Parameters
    --------------
    img : (ndarray) cv2処理されたオリジナル画像のndarray配列。
                    Channelは(B, G, R)。
    box : (tensor) リスケール画像のbboxの情報　[x1, y1, x2, y2]。
                   x1,y1はbboxの左上のx,y座標、x2, y2は右下のx,y座標を表す。
    label : (str) bboxに付与されるテキスト "bboxのクラス名 クラス確率(下2桁)"の文字列。
    color : (tuple | list-like) bboxの枠の色 (B, G, R)。
    txt_color : (tuple | list-like) bboxの付与するテキストの色。 
                                    PIL処理のため、Channelは(R, G, B)。

    
    Return
    -----------
    output : (ndarray) オリジナル画像にbbox(テキスト付き)を描画したndarray配列。
    
    """

    
    # YOLOv5の使用より (参照：ultralytics/ultralytics/utils/plotting.py)
    # line_widthは　1　と　2で挙動を確認済み
    line_width = max(math.floor(sum(img.shape) / 2 * 0.003), 2) # bboxの枠用に調整
    
    # p1:(x1, y1), p2:(x2, y2)
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    
    # bboxの描画
    cv2.rectangle(img, p1, p2, color, thickness=line_width, lineType=cv2.LINE_AA)
    
    if label:
        
        # 日本語フォントファイルのパス
        font_path = "font/Koruri-Bold.ttf"
    
        # 日本語フォントを読み込み
        font_size = int(line_width * 6)
        font = ImageFont.truetype(font_path, font_size)
    
        # テキストのサイズを計算
        text_bbox = font.getbbox(label) # return -> bboxの(left, top, right, bottom) 
        w, h = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
    
        # テキスト用の領域描画
        outside = p1[1] - h >= 3
        p2 = p1[0] + w + 7, p1[1] - h - 5 if outside else p1[1] + h + 5 # テキスト分の右上座標を計算 
        cv2.rectangle(img, p1, p2, color, -1, cv2.LINE_AA)  # テキスト領域の塗り潰し
    
        #------------ PIL (BGR -> RGB)　-------------
        # PILイメージに変換
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB変換
        img_pil = Image.fromarray(img)
        draw = ImageDraw.Draw(img_pil)
    
        # テキストを描画
        position = (p1[0]+ 4, p2[1] if outside else p1[1] + h + 2)
        draw.text(position, label, font=font, fill=txt_color)
    
        # ndarrayに変換
        img = np.array(img_pil)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGRに戻す
        

    return img

In [17]:
def scale_boxes(scaled_img_shape, boxes, img_shape):
    """
    渡されたbboxをオリジナル画像用に調整する。

    Parameters
    --------------
    scaled_img_shape : (tuple) リスケールした画像のshape()
    boxes : (tensor) リスケール画像用bboxの情報　[x1, y1, x2, y2]
                     x1,y1はbboxの左上のx,y座標、x2, y2は右下のx,y座標を表す
    img_shape : (tuple) オリジナル画像のshape, (H, W, C)の順番

    
    Return
    -----------
    output : (tensor) オリジナル画像用に調整したbboxの情報 [x1, y1, x2, y2]
                      オリジナル画像からはみ出る大きさのbboxはクリッピングされる。
    
    """
    
    ratio = min(scaled_img_shape[0] / img_shape[0], scaled_img_shape[1] / img_shape[1]) # new / old
    pad = (scaled_img_shape[1] - img_shape[1] * ratio) / 2, (scaled_img_shape[0] - img_shape[0] * ratio) / 2 # 片面のpad計算 
    
    # 元画像用にbboxを調整するため,
    # 片方のpad分を引き、リスケールの比率で割る
    boxes[:, [0, 2]] -= pad[0] # x padding
    boxes[:, [1, 3]] -= pad[1] # y padding
    boxes /= ratio 
    
    # 0以上、オリジナル画像の幅、高さ以内にbboxが入るようにクリップ
    clip_boxes(boxes, img_shape)

    return boxes



In [18]:
def clip_boxes(boxes, shape):
    """
    bboxがオリジナル画像の範囲内に収まるようにクリッピング
    0以上もしくはオリジナル画像の幅、高さに制限する。
    
    Parameters
    --------------
    boxes : (tensor) オリジナル画像用に調整したbbox, [x1, y1, x2, y2],
                     x1,y1はbboxの左上のx,y座標、x2, y2は右下のx,y座標を表す
    shape : (tuple) オリジナル画像のshape, (H, W, C)の順番


    Return
    -----------
    output : None 
             しかし、クリッピングにより変更がある場合は、直接boxesに直接変更が加えられる。
    
    """
    
    # bboxがオリジナル画像の範囲内に収まるようにクリッピング
    boxes[..., 0].clamp_(0, shape[1])  # x1
    boxes[..., 1].clamp_(0, shape[0])  # y1
    boxes[..., 2].clamp_(0, shape[1])  # x2
    boxes[..., 3].clamp_(0, shape[0])  # y2


In [19]:
def non_max_suppression(
    prediction, 
    conf_thres=0.25,
    iou_thres=0.45,
    agnostic=False,
    labels=(),
    max_det=300,
    nm=0,
):
    """
    
    YOLOの予測結果をNMS処理する。

    Parameters
    --------------
    prediction : (tensor) テンソル化したyoloが予測した結果
    conf_thres : (float) バウンディングボックス(bbox)の信頼度スコアの閾値
    iou_thres : (float) IOUの閾値
    agnostic : (Bool) 重なった異なるクラスのbboxを同一のbboxにするか
    labels : (tuple or list-like) 画像内のラベルの情報
    max_det : (int) 最大のbbox数
    nm : (int) マスク数
    
    Return
    -----------
    output : (tensor) NMS処理後のbbox情報を含むテンソル、[x1, y1, x2, y2, クラス確率, cls_id]のカラムに変換されている
                      x1,y1はbboxの左上のx,y座標、x2, y2は右下のx,y座標を表す
                      
    """
    batch_size = prediction.shape[0] # バッチサイズ
    num_class = prediction.shape[2] - nm - 5 # クラス数
    bool_cnf = prediction[..., 4] > conf_thres # bbox毎に信頼度スコアがconf_thresより大きいかのbool
    
    max_wh = 7680  # 最大のbboxの幅、高さ
    max_nms = 30000  # torchvision.ops.nms()のための最大のbbox数
    time_limit = 0.5 + 0.05 * batch_size  # タイムリミット(s)
    
    start = time.time()
    mi = 5 + num_class
    output = [torch.zeros((0, 6 + nm), device=prediction.device)] * batch_size
    
    
    for idx, x in enumerate(prediction):
    
        # conf_thresより大きい信頼度スコアを持つbboxsを抽出
        x = x[bool_cnf[idx]]
    
        # labelsを持っていた時の処理
        if labels and len(labels[idx]):
                lb = labels[idx]
                v = torch.zeros((len(lb), num_class + nm + 5), device=x.device)
                v[:, :4] = lb[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(lb)), lb[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)
            
    
        # フィルター後のbboxがない場合
        if not x.shape[0]:
            continue
            
        x[:, 5:] *= x[:, 4:5] # 信頼度スコア*各cls確率
        box = xywh2xyxy(x[:, :4]) # [x1, y1, x2, y2]:bboxの左上と右下の座標に変換
        mask = x[:, mi:] # maskがなければ[]
    
        cls_prob, cls_id = x[:, 5:mi].max(1, keepdim=True) # 各bboxで確率の高いclsの値とcls_idを返す
    
        # [x1, y1, x2, y2, clsの確率, cls_id]のカラムの順に結合
        # clsの確率(本来のprob*信頼度スコア)がconf_thresより高いもののみ抽出
        x = torch.cat((box, cls_prob, cls_id.float(), mask), axis=1)[cls_prob.view(-1) > conf_thres]
        
        n = x.shape[0]
        if not n:
            continue
    
        # cls確率をキーにして降順に並び替え、且つ max_nmsを超えないようにする
        x = x[x[:, 4].argsort(descending=True)[:max_nms]] 
    
        # 異なるクラスのbboxを区別して評価するため、agnosticでクラス固有のbboxを作成
        c = x[:, 5:6] * (0 if agnostic else max_wh)
        boxes, scores = x[:, :4] + c, x[:, 4]
    
        # nms
        selected_idx = torchvision.ops.nms(boxes, scores, iou_thres)
        selected_idx = selected_idx[:max_det]

        output[idx] = x[selected_idx]
        finish = time.time()
        if (finish - start) > time_limit:
            break
            
    return output


In [20]:
def xywh2xyxy(x):
    """
    [x中心、y中心、bboxの幅, bboxの高さ] -> [bboxの左上のx、bboxの左上のy, bboxの右下のx, bboxの右下のy]
    
    """
    y = x.clone()
    y[..., 0] = x[..., 0] - x[..., 2] / 2  # bboxの左上のx
    y[..., 1] = x[..., 1] - x[..., 3] / 2  # bboxの左上のy
    y[..., 2] = x[..., 0] + x[..., 2] / 2  # bboxの右下のx
    y[..., 3] = x[..., 1] + x[..., 3] / 2  # bboxの右下のy
    return y

In [21]:
def letterbox(
    img,
    new_shape=(640, 640),
    color=(0, 0, 0),
    scaleup=True,
):
    """
    指定されたサイズに画像をリサイズを行う。

    Parameters
    --------------
    img : (ndarray) 画像のndarray配列
    new_shape : (tuple) リサイズしたい画像サイズ
    color : (tuple:(B, G, R)) パディングの色
    scaleup : (Bool) スケールアップを行う場合はTrue

    Return
    -----------
    padded_img : (ndarray) リサイズされた画像のndarray配列
    ratio : (tuple リサイズした比率
    (dw, dh) : (int:(横、高さ)) パディングした数値(横、高さ)
    """
    
    # アスペクト比を保った画像のリサイズ
    ori_shape = img.shape[:2] # [H, W, C] -> [H, W]
    r = min(new_shape[0] / ori_shape[0], new_shape[1] / ori_shape[1])
    
    # スケールアップなしの場合
    if not scaleup:
        r = min(r, 1.0)
    
    # アスペクト比を保ったまま、リスケール
    ratio = (r, r)
    new_unpad = (round(ori_shape[1] * r), round(ori_shape[0] * r)) # cv.resizeのため[W, H]にする
    
    # パディングする領域を算出
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]
    dw, dh = dw / 2, dh / 2

    # dw, dfが.5の時の対応 -> この微調整をしないとsession.run()時のinputのshapeが合わなくなる
    top, bottom = round(dh - 0.1), round(dh + 0.1)
    left, right = round(dw - 0.1), round(dw + 0.1)
    
    # 画像のリサイズ
    if new_shape[::-1] != new_unpad:
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
        
    padded_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    return padded_img, ratio, (dw, dh)